In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [2]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

In [3]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [4]:
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [6]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1112, -1.7011]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1632], device='cuda:0', requires_grad=True)], Cost : 0.246
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1098, -1.7027]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1073], device='cuda:0', requires_grad=True)], Cost : 0.245
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1092, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0562], device='cuda:0', requires_grad=True)], Cost : 0.215
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1086, -1.7027]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0094], device='cuda:0', requires_grad=True)], Cost : 0.186
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1076, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0335], device='cuda:0', requires_grad=Tr

In [7]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.6007],
        [ 69.3008],
        [357.1305]], device='cuda:0')


In [9]:
torch.save(
    model,
    "../models/model.pt"
)

In [10]:
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)